In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import cv2
import numpy as np
from PIL import Image

from mtcnn.detector import detect_faces, get_net
from mtcnn.visualization_utils import show_bboxes
import mipkit
import matplotlib.pyplot as plt
import os
import glob
from align_faces import warp_and_crop_face

In [ ]:
def crop_faces(inputs):
    video_paths, save_dir, skip, crop_size = inputs
    
    # init model
    net = get_net()
    
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        basename = os.path.basename(video_path).split('.')[0]
        ext = os.path.basename(video_path).split('.')[1]
        
        if total_frames < 150:
            skip = 5
        
        if total_frames < 120:
            skip = 4
            
        if total_frames < 100:
            skip = 3
            
        for i in range(total_frames):
            if i % skip == 0:
                # Write to file
                ret, frame = cap.read()
                cap.set(1, i+1)
                bounding_boxes, landmarks = detect_faces(frame,
                                                         min_face_size=50.0,
                                                         thresholds=[0.5, 0.6, 0.7],
                                                         nms_thresholds=[0.6, 0.6, 0.7],
                                                         net=net)
                for lm in landmarks:
                    crop_face = warp_and_crop_face(frame, np.array(lm).reshape([2, 5]), 
                                       crop_size=crop_size, 
                                       default_square=True, 
                                       outer_padding=(0, 0))

                    folder_to_save = os.path.join(save_dir, basename + '_' + str(total_frames))
                    os.makedirs(folder_to_save, exist_ok=True)
                    path_to_save = os.path.join(*[folder_to_save,  
                                                  basename + '_' + mipkit.convert_int_to_str(i) + '.jpg'])
                    cv2.imwrite(path_to_save, crop_face)

        with open('processed_video.txt', 'a') as f:
            f.write(video_path + '\n')

# Load training dataset
load_dir = '/mnt/hdd4T/MERC2020/2020-1//val/*.mp4'
video_file_path = sorted(glob.glob(load_dir))

splitted_paths = mipkit.split_seq(video_file_path)

len(splitted_paths)

save_dir = '/mnt/hdd4T/MERC2020/2020-1/processed_data/val'
skip = 6
crop_size = 128, 128
inputs = [[spl, save_dir, skip, crop_size] for spl in splitted_paths]

mipkit.multiprocessing.pool_worker(target=crop_faces, inputs=inputs, num_worker=5)

In [29]:
# Load training dataset
load_dir = '/mnt/hdd4T/MERC2020/2020-1/val/*.mp4'
video_file_path = sorted(glob.glob(load_dir))

In [30]:
len(video_file_path)

5386

In [31]:
with open('processed_video.txt', 'r') as f:
    processed_paths = f.readlines()
    
processed_paths = set(map(lambda x: x.strip().replace('//', '/').replace('\n', '') , processed_paths))

In [32]:
video_file_path = set(video_file_path).difference(processed_paths)

In [33]:
len(video_file_path)

5224